In [0]:
import iarray_spark as iaspark
import iarray as ia
import numpy as np
from time import time

/databricks/python/lib/python3.8/site-packages/llvmlite/llvmpy/__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
 warnings.warn(
/databricks/python/lib/python3.8/site-packages/llvmlite/llvmpy/core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
 warnings.warn(

In [0]:
shape = (16 * 1024 * 16 * 1024,)
chunks = (2 * 1024 * 2 * 1024,)
blocks = (256 * 256,)
dtype = np.float64

urlpath_a = "iaspark_simple_a.iarr"
urlpath_b = "iaspark_simple_b.iarr"
urlpath_out = "iaspark_simple_out.iarr"

urlpath_dbfs_a = "/dbfs/iaspark_simple_a.iarr"
urlpath_dbfs_b = "/dbfs/iaspark_simple_b.iarr"

ia.remove_urlpath(urlpath_a)
ia.remove_urlpath(urlpath_b)
ia.remove_urlpath(urlpath_out)

ia.set_config_defaults(chunks=chunks, blocks=blocks, dtype=dtype, contiguous=True)

Out[2]: Config(codec=<Codec.LZ4: 1>, zfp_meta=0, clevel=9, favor=<Favor.BALANCE: 0>, filters=[<Filter.SHUFFLE: 1>], fp_mantissa_bits=0, use_dict=False, nthreads=8, eval_method=<Eval.AUTO: 1>, seed=1, random_gen=<RandomGen.MRG32K3A: 0>, btune=True, dtype=<class 'numpy.float64'>, np_dtype=None, split_mode=<SplitMode.AUTO_SPLIT: 3>, chunks=(4194304,), blocks=(65536,), urlpath=None, mode='w-', contiguous=True)

In [0]:
ia.arange(shape, urlpath=urlpath_a)
ia.arange(shape, urlpath=urlpath_b)

import shutil

shutil.copyfile(urlpath_a, urlpath_dbfs_a)
shutil.copyfile(urlpath_b, urlpath_dbfs_b)

Out[3]: '/dbfs/iaspark_simple_b.iarr'

In [0]:
expr = "tan(a + 2 * sin(b))"
expr = "2 * a + b"

In [0]:
# Now, it is using views (iterchunk engine), in order to use the iterblosc engine: 
#
# 1. Create an array from a chunk (chunk-array)
# 2. Eval using iterblosc
# 3. Serialize the chunk-array and send it to the server
# 4. Create an empty array and update the chunks
#
# or
#
# 1. Create an array from a chunk (chunk-array)
# 2. Eval using iterblosc

n = 3

t0 = time()
for i in range(n):
    print(i)
    res = iaspark.eval_split(expr, {"a": urlpath_a, "b": urlpath_b}, spark_context=sc, urlpath=urlpath_out)
t1 = time()

print(f"iarray optimized split array: {(t1 - t0)/n:.4f} s")

ia.remove_urlpath(urlpath_out)

t0 = time()
for i in range(n):
    print(i)
    res = iaspark.eval_opt(expr, {"a": urlpath_dbfs_a, "b": urlpath_dbfs_b}, spark_context=sc, urlpath=urlpath_out)
t1 = time()

print(f"iarray optimized shared filesystem: {(t1 - t0)/n:.4f} s")
ia.remove_urlpath(urlpath_out)



0
1
2
iarray optimized split array: 35.6456 s
0
1
2
iarray optimized shared filesystem: 9.9222 s

In [0]:
t0 = time()

a = ia.open(urlpath_dbfs_a)
b = ia.open(urlpath_dbfs_b)

res = ia.expr_from_string(expr, {"a": a, "b": b}).eval()
t1 = time()

print(t1 - t0)

8.454141616821289

In [0]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import shutil
from time import time


nelem = 16 * 16 * 1024 * 1024
chunks = 2 * 2 * 1024 * 1024
dtype = np.int64

urlpath_parquet = "/bench.parquet"
urlpath_dbfs_parquet = "/dbfs/bench.parquet"
urlpath_dbfs_out = "/bench_out.parquet"

try:
    os.remove(urlpath_parquet)
    dbutils.fs.rm(urlpath_dbfs_parquet, True)
    dbutils.fs.rm(urlpath_dbfs_out, True)
except:
    pass

np_arr = np.arange(nelem, dtype=dtype)
pa_table = pa.table({"data": np_arr})
pq.write_table(pa_table, row_group_size=chunks, where=urlpath_parquet)
shutil.copyfile(urlpath_parquet, urlpath_dbfs_parquet)


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

n = 3

t0 = time()
for i in range(n):
    print(i)
    spark = SparkSession.builder \
        .master('local') \
        .appName('myAppName') \
        .getOrCreate()

    parquetFile = spark.read.parquet(urlpath_parquet)

    parquetFile = parquetFile.withColumn("data", col("data") * lit(2))

    parquetFile.write.parquet(urlpath_out)
    try:
        dbutils.fs.rm(urlpath_out, True)
    except:
        pass
t1 = time()

print(f"time parquet: {(t1 - t0)/n:.4f}")

try:
    os.remove(urlpath_parquet)
    dbutils.fs.rm(urlpath_dbfs_parquet, True)
    dbutils.fs.rm(urlpath_out, True)
except:
    pass


0
1
2
time parquet: 14.8319